# Data loading
Main source: [eurostat](https://ec.europa.eu/eurostat/web/main/home)

*   [Bi-annual gas prices for houssehold consumers from 2007](https://appsso.eurostat.ec.europa.eu/nui/show.do?dataset=nrg_pc_202&lang=en)
*   [Bi-annual gas prices for non-household consumers from 2007](https://appsso.eurostat.ec.europa.eu/nui/show.do?dataset=nrg_pc_203&lang=en)
*   [Bi-annual data for domestic consumers until 2007](https://appsso.eurostat.ec.europa.eu/nui/show.do?dataset=nrg_pc_202_h&lang=en)
*   [Bi-annual data for industrial consumers until 2007](https://appsso.eurostat.ec.europa.eu/nui/show.do?dataset=nrg_pc_203_h&lang=en)



In [1]:
# import libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from google.colab import files
from scipy.stats import mannwhitneyu
from matplotlib.pyplot import figure
import seaborn as sns
sns.set()
%matplotlib inline
!pwd

# Dataset names in https://ec.europa.eu/eurostat/web/main/data/database

HEATING_INDEX_DATA = 'nrg_chdd_a' 
RENEWABLE_ENERGY_SHARE_DATA = 'nrg_ind_ren'
PPP_DATA = "prc_ppp_ind" # Purchase Power Parity
HICP_DATA = "prc_hicp_aind" # Harmonized prices index (inflation)
ENERGY_TAXES = "env_ac_taxener" # Energy taxes by paying sector
GPD_DATA = "nama_10_gdp" # GDP
ELECTRICITY_PRODUCTION = "nrg_bal_peh" # Production of electricity and derived heat by type of fuel
GROSS_AVAILABLE_ENERGY = "nrg_bal_c"
GAS_OPERATIONS = "nrg_cb_gas" # Supply, transformation and consumption of gas
IMPORTS_DEPENDENCY = "nrg_ind_id" # Energy imports dependency
ENERGY_EFFICIENCY = "nrg_ind_eff" # Energy efficiency
SHARE_OF_FUELS = "nrg_ind_fecf" # Share of fuels in final energy consumption
BIOFUELS_PRODUCTION = "nrg_inf_lbpc" # Liquid biofuels production capacities
EL_HEAT_PRODUCTION_BY_PLANT = "nrg_ind_pehap" # Production of electricity and heat by autoproducers, by type of plant

datasets_names = [
    HEATING_INDEX_DATA, 
    RENEWABLE_ENERGY_SHARE_DATA,
    PPP_DATA,
    HICP_DATA,
    ENERGY_TAXES,
    GPD_DATA,
    ELECTRICITY_PRODUCTION,
    GROSS_AVAILABLE_ENERGY,
    GAS_OPERATIONS,
    IMPORTS_DEPENDENCY,
    ENERGY_EFFICIENCY,
    SHARE_OF_FUELS,
    BIOFUELS_PRODUCTION,
    EL_HEAT_PRODUCTION_BY_PLANT
    ]


# Remove data from previous session
!rm -r eurostat_data

# Create a directory to store data from eurostat
!mkdir eurostat_data

# Download data from eurostat
for dataset in datasets_names:
  output_gz_file = f'eurostat_data/{dataset}.tsv.gz'
  !wget -O {output_gz_file} https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/{dataset}.tsv.gz

  output_tsv_file = f'eurostat_data/{dataset}.tsv'
  !gunzip -c {output_gz_file} > {output_tsv_file}
  !rm {output_gz_file}

%ls eurostat_data

/content
rm: cannot remove 'eurostat_data': No such file or directory
--2023-01-21 20:47:04--  https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/nrg_chdd_a.tsv.gz
Resolving ec.europa.eu (ec.europa.eu)... 147.67.34.30, 147.67.210.30, 2a01:7080:14:100::666:30, ...
Connecting to ec.europa.eu (ec.europa.eu)|147.67.34.30|:443... connected.
HTTP request sent, awaiting response... 200 
Cookie coming from ec.europa.eu attempted to set domain to europa.eu/eurostat
Length: 8464 (8.3K) [application/octet-stream]
Saving to: ‘eurostat_data/nrg_chdd_a.tsv.gz’

eurostat_data/nrg_c 100%[===================>]   8.27K  --.-KB/s    in 0.001s  

2023-01-21 20:47:05 (8.92 MB/s) - ‘eurostat_data/nrg_chdd_a.tsv.gz’ saved [8464/8464]

--2023-01-21 20:47:05--  https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/nrg_ind_ren.tsv.gz
Resolving ec.europa.eu (ec.europa.eu)... 147.67.34.30, 147.67.210.30, 2a01:7080:14:100::666:30, ...
Connecting 

# Data preprocessing
## Convert eurostat data to data frames

In [2]:
heating_index_df = pd.read_csv(f'eurostat_data/{HEATING_INDEX_DATA}.tsv', sep='\t')
renewable_energy_share_df = pd.read_csv(f'eurostat_data/{RENEWABLE_ENERGY_SHARE_DATA}.tsv', sep='\t')
ppp_df = pd.read_csv(f'eurostat_data/{PPP_DATA}.tsv', sep='\t')
hicp_df = pd.read_csv(f'eurostat_data/{HICP_DATA}.tsv', sep='\t')
energy_taxes_df = pd.read_csv(f'eurostat_data/{ENERGY_TAXES}.tsv', sep='\t')
gdp_df = pd.read_csv(f'eurostat_data/{GPD_DATA}.tsv', sep='\t')
electricity_production_df = pd.read_csv(f'eurostat_data/{ELECTRICITY_PRODUCTION}.tsv', sep='\t')
available_energy_df = pd.read_csv(f'eurostat_data/{GROSS_AVAILABLE_ENERGY}.tsv', sep='\t')
gas_operations_df = pd.read_csv(f'eurostat_data/{GAS_OPERATIONS}.tsv', sep='\t')
imports_dependency_df = pd.read_csv(f'eurostat_data/{IMPORTS_DEPENDENCY}.tsv', sep='\t')
energy_efficiency_df = pd.read_csv(f'eurostat_data/{ENERGY_EFFICIENCY}.tsv', sep='\t')
share_of_fuels_df = pd.read_csv(f'eurostat_data/{SHARE_OF_FUELS}.tsv', sep='\t')
biofules_production_df = pd.read_csv(f'eurostat_data/{BIOFUELS_PRODUCTION}.tsv', sep='\t')
el_heat_production_by_plant_df = pd.read_csv(f'eurostat_data/{EL_HEAT_PRODUCTION_BY_PLANT}.tsv', sep='\t')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Filtering data from redundant values and columns

In [3]:
# We need to choose only one value for product, consom, unit, tax, and currency 
countries_map = {
                        "BE": "Belgium",
                    "BG": "Bulgaria",
                    "CZ": "Czechia",
                    "DK": "Denmark",
                    "DE": "Germany",
                    "EE": "Estonia",
                    "IE": "Ireland",
                    "EL": "Greece",
                    "ES": "Spain",
                    "FR": "France",
                    "HR": "Croatia",
                    "IT": "Italy",
                    "CY": "Cyprus",
                    "LV": "Latvia",
                    "LT": "Lithuania",
                    "LU": "Luxembourg",
                    "HU": "Hungary",
                    "MT": "Malta",
                    "NL": "Netherlands",
                    "AT": "Austria",
                    "PL": "Poland",
                    "PT": "Portugal",
                    "RO": "Romania",
                    "SI": "Slovenia",
                    "SK": "Slovakia",
                    "FI": "Finland",
                    "SE": "Sweden",
                    "IS": "Iceland",
                    "LI": "Liechtenstein",
                    "NO": "Norway",
                    "CH": "Switzerland",
                    "UK": "United Kingdom",
                    "ME": "Montenegro",
                    "MK": "North Macedonia",
                    "AL": "Albania",
                    "RS": "Serbia",
                    "TR": "Türkiye",
                    "AD": "Andorra",
                    "BY": "Belarus",
                    "BA": "Bosnia and Herzegovina",
                    "GI": "Gibraltar (UK)",
                    "XK": "Kosovo",
                    "MD": "Moldova",
                    "RU": "Russia",
                    "UA": "Ukraine",
                    "EX_SU_OTH": "Countries of former Soviet Union",
                    "EUR_OTH": "Other European countries",
                    "AO": "Angola",
                    "CM": "Cameroon",
                    "CG": "Congo",
                    "CD": "Democratic Republic of the Congo",
                    "GQ": "Equatorial Guinea",
                    "GA": "Gabon",
                    "ST": "São Tomé and Príncipe",
                    "DJ": "Djibouti",
                    "ER": "Eritrea",
                    "ET": "Ethiopia",
                    "KE": "Kenya",
                    "MG": "Madagascar",
                    "MU": "Mauritius",
                    "MZ": "Mozambique",
                    "UG": "Uganda",
                    "TZ": "Tanzania",
                    "DZ": "Algeria",
                    "EG": "Egypt",
                    "LY": "Libya",
                    "MA": "Morocco",
                    "SS": "South Sudan",
                    "SD": "Sudan",
                    "TN": "Tunisia",
                    "NA": "Namibia",
                    "ZA": "South Africa",
                    "BJ": "Benin",
                    "CV": "Cabo Verde",
                    "CI": "Côte d'Ivoire",
                    "GH": "Ghana",
                    "GW": "Guinea-Bissau",
                    "LR": "Liberia",
                    "MR": "Mauritania",
                    "NE": "Niger",
                    "NG": "Nigeria",
                    "SN": "Senegal",
                    "SL": "Sierra Leone",
                    "TG": "Togo",
                    "AFR_OTH": "Other African countries",
                    "CA": "Canada",
                    "US": "United States",
                    "AW": "Aruba (NL)",
                    "BS": "Bahamas",
                    "BB": "Barbados",
                    "VG": "British Virgin Islands (UK)",
                    "CU": "Cuba",
                    "CW": "Curaçao",
                    "DO": "Dominican Republic",
                    "JM": "Jamaica",
                    "TT": "Trinidad and Tobago",
                    "BZ": "Belize",
                    "CR": "Costa Rica",
                    "GT": "Guatemala",
                    "HN": "Honduras",
                    "MX": "Mexico",
                    "PA": "Panama",
                    "AR": "Argentina",
                    "BO": "Bolivia",
                    "BR": "Brazil",
                    "CL": "Chile",
                    "CO": "Colombia",
                    "EC": "Ecuador",
                    "PE": "Peru",
                    "UY": "Uruguay",
                    "VE": "Venezuela",
                    "AME_OTH": "Other American countries",
                    "KZ": "Kazakhstan",
                    "KG": "Kyrgyzstan",
                    "TJ": "Tajikistan",
                    "TM": "Turkmenistan",
                    "UZ": "Uzbekistan",
                    "CN": "China including Hong Kong",
                    "HK": "Hong Kong",
                    "JP": "Japan",
                    "MN": "Mongolia",
                    "KP": "North Korea",
                    "KR": "South Korea",
                    "TW": "Taiwan",
                    "BD": "Bangladesh",
                    "IN": "India",
                    "IR": "Iran",
                    "NP": "Nepal",
                    "PK": "Pakistan",
                    "LK": "Sri Lanka",
                    "BN": "Brunei Darussalam",
                    "KH": "Cambodia",
                    "ID": "Indonesia",
                    "LA": "Laos",
                    "MY": "Malaysia",
                    "MM": "Myanmar/Burma",
                    "PH": "Philippines",
                    "SG": "Singapore",
                    "TH": "Thailand",
                    "TL": "Timor-Leste",
                    "VN": "Vietnam",
                    "AM": "Armenia",
                    "AZ": "Azerbaijan",
                    "BH": "Bahrain",
                    "GE": "Georgia",
                    "IQ": "Iraq",
                    "IL": "Israel",
                    "JO": "Jordan",
                    "KW": "Kuwait",
                    "LB": "Lebanon",
                    "OM": "Oman",
                    "QA": "Qatar",
                    "SA": "Saudi Arabia",
                    "SY": "Syria",
                    "AE": "United Arab Emirates",
                    "YE": "Yemen",
                    "ASI_NME_OTH": "Other Near and Middle East Asian countries",
                    "ASI_OTH": "Other Asian countries",
                    "AU": "Australia",
                    "NZ": "New Zealand",
                    "NC": "New Caledonia (FR)",
                    "PG": "Papua New Guinea",
                    "MH": "Marshall Islands",
                    "TOTAL": "Total",
                    "NSP": "Not specified"
                }

# Imporant columns names
GEO = 'geo\\time'

# HDD - heating days index
# REN_HEAT_CL - renewable energy share for heating and cooling
# REN - renewable energy share
# A01 - ppp for everything
# A0104 - ppp for housing electricity, gas and other fuels
# CP00 - all items for hicp
# CP04 - housing electricity, gas and other fuels for hicp
# ELC_GAS - electricity, gas, solid fuels and heat energy for hicp
# EP_HH - energy taxes for households
# B-E energy taxes for Industry (except construction)
# B1GQ - gross domestic product at market prices 
# GHP - gross heat production
# GEP - gross electricity production
# GAE - gross available energy
# G3000 - natural gas
# TJ_GCV - tera joule
# IC_CAL - inland consumption
# PEC2020-2030 - Primary energy consumption (Europe 2020-2030) energy efficiency
# FC_E - final energy consumpltion in share of fuels
# FC_IND_E - final energy consumpltion in share of fuels industry sector
# FC_OTH_HH_E - final energy consumpltion in share of fuels domestic sector
# RA000 - Renewables and biofuels
# H8000 - Heat
# R5210P - Pure biogasoline
# R5220P - Pure biodiesels 
# PRD - production
# FC_IND_E - Final consumption - industry sector - energy use


bad_countries = [
                    "EU27_2020",
                    "EU28",
                    "EU27_2007",
                    "EU25",
                    "EU15",
                    "EA20",
                    "EA19",
                    "EA18",
                    "EA17",
                    "EA16",
                    "EA15",
                    "EA13",
                    "EA12",
                    "EA11",
                    "CPC1",
                    "XK",
                    "EA",
                    "EEA",
                    "EU",
                    "US",
                    "JP",
                    "BA",
                    "MK",
                    "ME",
                    "IS",
                    "CH",
                    "MT",
                    "EL",
                    "CY",
                    "AL",
                    "TR",
                    "RS",
                    "NO",
                    "UK",
                 "GE",
                 
]

domestic_clusters = [
    ["Belgium", "Bulgaria", "Czechia", "Denmark", "Estonia", "Netherlands", "Romania", "Sweden", "Luxembourg"],
    ["Austria", "Germany", "Spain", "France", "Croatia",  "Hungary", "Ireland", "Italy", "Lithuania", "Latvia", "Poland", "Portugal", "Slovenia", "Slovakia", "United Kingdom",]
]

industrial_clusters = [
    ["Belgium", "Germany", "United Kingdom", "Croatia", "Hungary", "Slovenia"],
    ["Austria", "Bulgaria", "Czechia", "Denmark", "Estonia", "Spain", "France", "Finland", "Italy", "Lithuania", "Latvia", "Poland", "Portugal", "Slovakia", "Netherlands", "Romania", "Sweden", "Luxembourg"]
]

class Config:
  def __init__(self,
               non_values_columns,
               columns_to_filter,
               filtering_values,
               should_drop_na,
               dates_to_remove = [],
               columns_to_drop = [],
               ):
    self.non_values_columns = non_values_columns
    self.columns_to_filter = columns_to_filter
    self.filtering_values = filtering_values
    self.should_drop_na = should_drop_na
    self.dates_to_remove = dates_to_remove
    self.columns_to_drop = columns_to_drop


heating_index_config = Config(
    [GEO, "unit", "indic_nrg"],
    ["unit", "indic_nrg"],
    {"unit": 'NR', "indic_nrg": 'HDD'},
    True,
    dates_to_remove=['2002 ', '1979 '],
)

renewable_energy_share_config = Config(
    [GEO, "nrg_bal", "unit"],
    ["nrg_bal"],
    {"nrg_bal": 'REN'},
    False,
)


ppp_config = Config(
    [GEO, "na_item", "ppp_cat"],
    ["na_item", "ppp_cat"],
    {"na_item": 'PPP_EU27_2020', "ppp_cat": 'A01'}, 
    False,
    dates_to_remove=['2002 ', '1995 '],
    # columns_to_drop=["Hungary"]
)


hicp_config = Config(
    [GEO, "unit", "coicop"],
    ["unit", "coicop"],
    {"unit": 'INX_A_AVG', "coicop": 'ELC_GAS'}, 
    False,
    dates_to_remove=['2002 ', '1996 ']
)


energy_taxes_households_config = Config(
    [GEO, "unit", "nace_r2"],
    ["unit", "nace_r2"],
    {"unit": 'PC', "nace_r2": 'EP_HH'}, 
    False,
    dates_to_remove=['2002 ', '1995 ']
)

energy_taxes_industry_config = Config(
    [GEO, "unit", "nace_r2"],
    ["unit", "nace_r2"],
    {"unit": 'PC', "nace_r2": 'B-E'}, 
    False,
    dates_to_remove=['2002 ', '1995 ']
)

gdp_config = Config(
    [GEO, "unit", "na_item"],
    ["unit", "na_item"],
    {"unit": 'CP_MEUR', "na_item": 'B1GQ'}, 
    False,
    dates_to_remove=['2002 ', '1975 ']
)

electricity_production_config = Config(
    [GEO, "unit", "nrg_bal", "siec"],
    ["unit", "nrg_bal", "siec"],
    {"unit": 'GWH', "nrg_bal": 'GEP', "siec": "G3000"}, 
    False,
    dates_to_remove=['2002 ', '1990 ']
)

available_energy_config = Config(
    [GEO, "unit", "nrg_bal", "siec"],
    ["unit", "nrg_bal", "siec"],
    {"unit": 'GWH', "nrg_bal": 'GAE', "siec": "G3000"}, 
    False,
    dates_to_remove=['2002 ', '1990 ']
)

gas_operations_config = Config(
    [GEO, "unit", "nrg_bal", "siec"],
    ["unit", "nrg_bal", "siec"],
    {"unit": 'TJ_GCV', "nrg_bal": 'IC_CAL', "siec": "G3000"}, 
    False,
    dates_to_remove=['2002 ', '1990 ']
)

imports_dependency_config = Config(
    [GEO, "unit", "siec"],
    ["siec"],
    {"siec": "TOTAL"}, 
    False,
    dates_to_remove=['2002 ', '1990 ']
)

energy_efficiency_config = Config(
    [GEO, "unit", "nrg_bal"],
    ["unit", "nrg_bal"],
    {"unit": 'MTOE', "nrg_bal": "PEC2020-2030"}, 
    False,
    dates_to_remove=['2002 ', '1990 ']
)

share_of_fuels_renewables_config = Config(
    [GEO, "unit", "nrg_bal", "siec"],
    ["nrg_bal", "siec"],
    {"nrg_bal": "FC_IND_E",  "siec": "RA000"}, 
    False,
    dates_to_remove=['2004 ', '1990 ']
)

share_of_fuels_heat_config = Config(
    [GEO, "unit", "nrg_bal", "siec"],
    ["nrg_bal", "siec"],
    {"nrg_bal": "FC_OTH_HH_E",  "siec": "H8000"}, 
    False,
    dates_to_remove=['2002 ', '1990 ']
)

biofules_production_config = Config(
    [GEO, "unit", "siec", "plant_tec"],
    ["siec"],
    {"siec": "R5220P"}, 
    False,
    dates_to_remove=['1994 ', '1990 ']
)

el_heat_production_by_plant_config = Config(
    [GEO, "unit", "plants", "siec", "operator", "nrg_bal"],
    ["unit", "plants", "siec", "nrg_bal"],
    {"plants": "TOTAL", "siec": "H8000", "unit": 'TJ', "nrg_bal": "FC_IND_E",}, 
    False,
    dates_to_remove=['2002 ', '1990 ']
)


## Preprocessing

In [4]:
def split_column(df, column_to_split):
  new_columns_names = column_to_split.split(',')

  first_cols = df.iloc[:, 0].str.split(',', expand = True)

  old_new_names_map = dict(zip(first_cols.columns, new_columns_names))
  first_cols.rename(columns=old_new_names_map, inplace=True)

  df = pd.concat([df, first_cols], axis=1)
  df.drop(columns = df.columns[0], axis = 1, inplace= True)
  return df


def identify_na(df, replacements):
  for key, value in replacements.items():
    for column in df.columns:
      df[column] = df[column].replace(key, value)
  return df

# Filter values (choosing one tax category, one consumption category etc)
def filter_columns(df, columns, filtering_values):
  for column in columns:
    value = filtering_values[column]
    if type(value) == np.ndarray or type(value) == list:
      df = df[df[column].isin(value)]
    else:
      df = df[df[column] == value]
  return df



def preprocess_data(df, config: Config):
  df = split_column(df, df.iloc[:, 0].name)
  df = identify_na(df, {': ': np.nan})
  df = identify_na(df, {': c': np.nan})

  # print(df.columns)
  # print(df.value_counts())

  df = filter_columns(df, config.columns_to_filter, config.filtering_values)

  # remove bad countries
  df = df[~df[GEO].isin(bad_countries)]

  # Add column "country", which contains full country name
  COUNTRY = 'country'
  df[COUNTRY] = df[GEO].apply(lambda x: countries_map[x])

  # remove extra columns
  reduced_df = df.drop(config.non_values_columns, axis=1)


  # drop few columns
  if (len(config.dates_to_remove) != 0):
    reduced_df.drop(df.loc[:, config.dates_to_remove[0] : config.dates_to_remove[1]].columns, axis=1, inplace=True)

  # drop na
  if (config.should_drop_na):
    cleaned_df = reduced_df.dropna(axis=1)
  else:
    cleaned_df = reduced_df.fillna(method='bfill', axis=0)
    # drop row if it has 70% of na
    threshhold = len(cleaned_df.columns) * 0.7
    cleaned_df.dropna(axis=1, thresh=threshhold, inplace=True)

    # fill the rest
    cleaned_df.fillna(method ='ffill', inplace=True)
    # cleaned_df = reduced_df.fillna(method='bfill', axis=1)


  # print(cleaned_df)

  # set index as countries
  cleaned_df = cleaned_df.set_index(cleaned_df.country)
  cleaned_df = cleaned_df.drop("country", axis=1)

  # convert to float
  cleaned_df = cleaned_df.apply(lambda x: x.apply(lambda x: x.split(' ')[0] if not isinstance(x, float) else x).astype(float))

  # drop some countires
  if (len(config.columns_to_drop) != 0):
    for c in config.columns_to_drop:
      cleaned_df = cleaned_df.drop(index=(c))

  return cleaned_df

## Mann Whitney U-test

In [5]:
def calculate_mw_test(regressor, regressor_name, source):
  cluster_1 = regressor.filter(items=source[0])
  cluster_2 = regressor.filter(items=source[1])
  
  U1, p = mannwhitneyu(cluster_1, cluster_2, method="exact")
  return U1, p


def get_u_test_results(df, config):
  df_to_mutate = df.copy()
  proceed_df = preprocess_data(
      df_to_mutate,
      config
  )

  transposed_df = proceed_df.transpose()
  median = transposed_df.median()
  std = transposed_df.std()
  mean = transposed_df.mean()

  figure(figsize=(20, 10), dpi=80)

  U1_median, p_median = calculate_mw_test(median, "median", domestic_clusters)
  U1_mean, p_mean = calculate_mw_test(mean, "mean", domestic_clusters)
  U1_std, p_std = calculate_mw_test(std, "std", domestic_clusters)

  d_domestic = {'value': ['median', 'mean', 'std'],
      'U1': [U1_median, U1_mean, U1_std],
      'p': [p_median, p_mean, p_std]}

  U1_median, p_median = calculate_mw_test(median, "median", industrial_clusters)
  U1_mean, p_mean = calculate_mw_test(mean, "mean", industrial_clusters)
  U1_std, p_std = calculate_mw_test(std, "std", industrial_clusters)

  d_industrial = {'value': ['median', 'mean', 'std'],
  'U1': [U1_median, U1_mean, U1_std],
  'p': [p_median, p_mean, p_std]}

  df_domestic = pd.DataFrame(data=d_domestic)
  df_domestic.name = "Domestic"
  df_industrial = pd.DataFrame(data=d_industrial)
  df_industrial.name = "Industrial"

  return df_domestic, df_industrial

## Results

In [6]:
# heating index

df1, df2 = get_u_test_results(heating_index_df, heating_index_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value    U1         p
0  median  78.0  0.368580
1    mean  78.0  0.368580
2     std  89.0  0.109301


Industrial
    value    U1         p
0  median  29.0  0.257066
1    mean  28.0  0.226812
2     std  33.0  0.403281




<Figure size 1600x800 with 0 Axes>

In [7]:
# renewable energy share (all sectors)

df1, df2 = get_u_test_results(renewable_energy_share_df, renewable_energy_share_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value    U1         p
0  median  59.0  0.829075
1    mean  56.0  0.688251
2     std  73.0  0.557077


Industrial
    value    U1         p
0  median  35.0  0.490951
1    mean  33.0  0.403281
2     std  18.0  0.045707




<Figure size 1600x800 with 0 Axes>

In [8]:
# purchasing power parity (housing electricity, gas and other fuels)

df1, df2 = get_u_test_results(ppp_df, ppp_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value     U1         p
0  median   95.0  0.045585
1    mean   95.0  0.045585
2     std  100.0  0.019131


Industrial
    value    U1         p
0  median  41.0  0.800618
1    mean  43.0  0.913846
2     std  30.0  0.289934




<Figure size 1600x800 with 0 Axes>

In [9]:
# harmonized index of counsumer prices (electricity, gas, solid fuels and heat energy)

df1, df2 = get_u_test_results(hicp_df, hicp_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value    U1         p
0  median  92.5  0.072093
1    mean  99.0  0.023015
2     std  75.0  0.476707


Industrial
    value    U1         p
0  median  59.0  0.325181
1    mean  43.0  0.913846
2     std  41.0  0.800618




<Figure size 1600x800 with 0 Axes>

In [10]:
# energy taxes (domestic, industry)

df1, _ = get_u_test_results(energy_taxes_df, energy_taxes_households_config)
_, df2 = get_u_test_results(energy_taxes_df, energy_taxes_industry_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value    U1         p
0  median  41.0  0.179361
1    mean  45.0  0.276560
2     std  57.0  0.734304


Industrial
    value    U1         p
0  median  44.0  0.971262
1    mean  44.0  0.971262
2     std  46.0  0.971262




<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

In [11]:
# gross domestic product at market prices (all sectros)

df1, df2 = get_u_test_results(gdp_df, gdp_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value    U1         p
0  median  57.0  0.734304
1    mean  57.0  0.734304
2     std  55.0  0.643258


Industrial
    value    U1         p
0  median  44.0  0.971262
1    mean  44.0  0.971262
2     std  41.0  0.800618




<Figure size 1600x800 with 0 Axes>

In [12]:
# Production of electricity and derived heat by type of fuel (natural gas)

df1, df2 = get_u_test_results(electricity_production_df, electricity_production_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value    U1         p
0  median  40.0  0.159446
1    mean  41.0  0.179361
2     std  46.0  0.305405


Industrial
    value    U1         p
0  median  53.0  0.587358
1    mean  51.0  0.691135
2     std  40.0  0.745342




<Figure size 1600x800 with 0 Axes>

In [13]:
# Gross available energy (natural gas)

df1, df2 = get_u_test_results(available_energy_df, available_energy_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value    U1         p
0  median  47.0  0.336066
1    mean  47.0  0.336066
2     std  53.0  0.557077


Industrial
    value    U1         p
0  median  50.0  0.745342
1    mean  50.0  0.745342
2     std  41.0  0.800618




<Figure size 1600x800 with 0 Axes>

In [14]:
# Supply, transformation and consumption of gas (natural gas)

df1, df2 = get_u_test_results(gas_operations_df, gas_operations_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value    U1         p
0  median  47.0  0.336066
1    mean  50.0  0.438946
2     std  72.0  0.599511


Industrial
    value    U1         p
0  median  50.0  0.745342
1    mean  47.0  0.913846
2     std  35.0  0.490951




<Figure size 1600x800 with 0 Axes>

In [15]:
# Energy imports dependency (all energy sources)

df1, df2 = get_u_test_results(imports_dependency_df, imports_dependency_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value    U1         p
0  median  29.0  0.032746
1    mean  30.0  0.038745
2     std  77.0  0.402871


Industrial
    value    U1         p
0  median  58.0  0.363042
1    mean  57.0  0.403281
2     std  16.0  0.030253




<Figure size 1600x800 with 0 Axes>

In [16]:
# Energy efficiency (Primary energy consumption (Europe 2020-2030) energy efficiency)

df1, df2 = get_u_test_results(energy_efficiency_df, energy_efficiency_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value    U1         p
0  median  58.0  0.781314
1    mean  58.0  0.781314
2     std  64.0  0.975411


Industrial
    value    U1         p
0  median  47.0  0.913846
1    mean  46.0  0.971262
2     std  41.0  0.800618




<Figure size 1600x800 with 0 Axes>

In [17]:
# Share of fuels in final energy consumption (renewables and biofuels in industry sector)
 
df1, df2 = get_u_test_results(share_of_fuels_df, share_of_fuels_renewables_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value    U1         p
0  median  57.0  0.734304
1    mean  54.0  0.599511
2     std  72.0  0.599511


Industrial
    value    U1         p
0  median  18.0  0.045707
1    mean  20.0  0.066688
2     std  25.0  0.150435




<Figure size 1600x800 with 0 Axes>

In [18]:
# Share of fuels in final energy consumption (heat for households sector)
 
df1, df2 = get_u_test_results(share_of_fuels_df, share_of_fuels_heat_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value    U1         p
0  median  80.0  0.305405
1    mean  81.0  0.276560
2     std  77.0  0.402871


Industrial
    value    U1         p
0  median  27.0  0.198936
1    mean  27.0  0.198936
2     std  17.0  0.037327




<Figure size 1600x800 with 0 Axes>

In [19]:
# Liquid biofuels production capacities (Pure biodiesels)
 
df1, df2 = get_u_test_results(biofules_production_df, biofules_production_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value    U1         p
0  median  31.0  0.045585
1    mean  37.0  0.109301
2     std  46.0  0.305405


Industrial
    value    U1         p
0  median  44.0  0.971262
1    mean  40.0  0.745342
2     std  40.0  0.745342




<Figure size 1600x800 with 0 Axes>

In [20]:
# Production of electricity and heat by autoproducers (heat, all kinds of plants, for industry sector)

df1, df2 = get_u_test_results(el_heat_production_by_plant_df, el_heat_production_by_plant_config)
print(df1.name)
print(df1)
print("\n")

print(df2.name)
print(df2)
print("\n")

Domestic
    value    U1         p
0  median  75.0  0.476707
1    mean  74.0  0.516127
2     std  79.0  0.336066


Industrial
    value    U1         p
0  median  19.0  0.055395
1    mean  18.0  0.045707
2     std  23.0  0.110910




<Figure size 1600x800 with 0 Axes>

## Logistic regression

In [21]:
import statsmodels.api as sm

def get_metrics(df, config):
  df_to_mutate = df.copy()
  proceed_df = preprocess_data(
      df_to_mutate,
      config
  )

  transposed_df = proceed_df.transpose()
  median = transposed_df.median()
  std = transposed_df.std()
  mean = transposed_df.mean()
  return median, mean, std

def get_logit_summary(metrics, clusters):
  log_reg = sm.Logit(clusters, metrics).fit()
  return log_reg.summary()

def get_metrics_and_labels(clusters, indicators):
  metrics = []
  clusters_labels = []
  for i, _ in enumerate(clusters):
    m = []
    l = []
    for index, country in enumerate(clusters[i]):
      l.append(i)
      m.append([])
      for j, indicator in enumerate(indicators):
        m[index].append(indicator[country])
    metrics += m
    clusters_labels += l
  return metrics, clusters_labels


In [22]:
_, _, renewable_share_std = get_metrics(renewable_energy_share_df, renewable_energy_share_config)
ppp_median, ppp_mean, ppp_std = get_metrics(ppp_df, ppp_config)
imports_dependency_median, imports_dependency_mean, imports_dependency_std = get_metrics(imports_dependency_df, imports_dependency_config)
_, _, share_of_fuels_renewables_std = get_metrics(share_of_fuels_df, share_of_fuels_renewables_config)
_, el_heat_production_by_plant_mean, _ = get_metrics(el_heat_production_by_plant_df, el_heat_production_by_plant_config)
biofuels_production_median, _, _ = get_metrics(biofules_production_df, biofules_production_config)
_, hicp_mean, _ = get_metrics(hicp_df, hicp_config)

In [23]:
# Industrial
industrial_good_indicators = [
    renewable_share_std,
    imports_dependency_std,
    share_of_fuels_renewables_std,
    el_heat_production_by_plant_mean,
]

cl1 = np.copy(industrial_clusters[0])
cl2 = np.copy(industrial_clusters[1])
industrials_clusters_no_uk = [cl1.tolist(), cl2.tolist()]

# some indicators do not have UK
industrials_clusters_no_uk[0].remove('United Kingdom')
industrial_metrics_by_country, industrial_clusters_labels = get_metrics_and_labels(industrials_clusters_no_uk, industrial_good_indicators)
df = pd.DataFrame(industrial_metrics_by_country, columns=['renewable_share_std', "imports_dependency_std", "share_of_fuels_renewables_std", "heat_production_by_plant_mean"])

summary = get_logit_summary(df, industrial_clusters_labels)
summary

Optimization terminated successfully.
         Current function value: 0.322142
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   23
Model:                          Logit   Df Residuals:                       19
Method:                           MLE   Df Model:                            3
Date:                Sat, 21 Jan 2023   Pseudo R-squ.:                  0.3847
Time:                        20:48:03   Log-Likelihood:                -7.4093
converged:                       True   LL-Null:                       -12.042
Covariance Type:            nonrobust   LLR p-value:                   0.02595
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
renewable_share_std              -0.6415      0.554     -1.158      0.247      -1.727       0.444
imports_dependency_std            0.1979      0.259      0.765      0.444      -0.309       0.705
share_of_fuels_renewables_std     1.5037      1.372      1.096      0.273      -1.185       4.193
heat_production_by_plant_mean     0.0002      0.000      1.185      0.236      -0.000       0.001
=================================================================================================
"""

In [24]:
# Domestic
domestic_good_indicators = [
    ppp_median,
    ppp_std,
    hicp_mean,
    biofuels_production_median,
    imports_dependency_median,
]

cl1 = np.copy(domestic_clusters[0])
cl2 = np.copy(domestic_clusters[1])
domestic_clusters_no_uk = [cl1.tolist(), cl2.tolist()]

# some indicators do not have UK
domestic_clusters_no_uk[1].remove('United Kingdom')

domestic_metrics_by_country, domestic_clusters_labels = get_metrics_and_labels(domestic_clusters_no_uk, domestic_good_indicators)
df = pd.DataFrame(domestic_metrics_by_country, columns=['ppp_median', "ppp_std", "hicp_mean", "biofuels_production_median", "imports_dependency_median"])

summary = get_logit_summary(df, domestic_clusters_labels)
summary

Optimization terminated successfully.
         Current function value: 0.391082
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   23
Model:                          Logit   Df Residuals:                       18
Method:                           MLE   Df Model:                            4
Date:                Sat, 21 Jan 2023   Pseudo R-squ.:                  0.4157
Time:                        20:48:03   Log-Likelihood:                -8.9949
converged:                       True   LL-Null:                       -15.395
Covariance Type:            nonrobust   LLR p-value:                   0.01230
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
ppp_median                    -0.4981      0.854     -0.583      0.560      -2.172       1.176
ppp_std                        3.8804      6.338      0.612      0.540      -8.541      16.302
hicp_mean                     -0.0235      0.019     -1.264      0.206      -0.060       0.013
biofuels_production_median     0.0026      0.002      1.111      0.267      -0.002       0.007
imports_dependency_median      0.0400      0.026      1.522      0.128      -0.011       0.091
==============================================================================================
"""